In [1]:
import json
import pprint

Посмотрим на то, какие ошибки есть в тесте с simulated тулами

In [3]:
test_generations_real_7b = json.load(open('eval_generations/eval_7b_testv1_simulated/api_data_0_10.json', 'r'))

In [4]:
error_count = 0

for item in test_generations_real_7b:
    for instance in item['Instances']:
        if 'error' in instance:
            error_count += 1
            pprint.pprint(instance)
            print('#' * 100)

{'error': 'can only concatenate str (not "list") to str'}
####################################################################################################
{'error': 'Could not parse LLM output: ` `'}
####################################################################################################
{'error': 'Could not parse LLM output: ` I need.`'}
####################################################################################################


In [5]:
error_count

3

In [6]:
test_generations_real_7b[0]['Instances'][0]

{'input': 'Hey, can you show me a random picture of an axolotl?',
 'output': 'Here is a random picture of an axolotl: https://theaxolotlapi.netlify.app/images/axolotl5.jpg. It is a beautiful axolotl with unique markings, and is from the Unsplash collection.',
 'Final Thought': 'The getRandomAxolotlImage tool successfully retrieved a random axolotl image with information on the image source.',
 'intermediate_steps': [[['getRandomAxolotlImage',
    '{}',
    ' I can use the getRandomAxolotlImage tool to retrieve a random axolotl image.\nASSISTANT Action: getRandomAxolotlImage\nASSISTANT Action Input: {}'],
   'Status Code: 200. Response: {"url":"https://theaxolotlapi.netlify.app/images/axolotl5.jpg","source":"Unsplash","description":"A beautiful axolotl with unique markings"}']]}

In [14]:
error_count = 0
count_not_status_code = 0
count_status_code = 0
all_count = 0
error_status_code = 0
error_not_status_code = 0
for item in test_generations_real_7b:
    for instance in item['Instances']:
        if 'error' in instance:
            error_count += 1
            all_count += 1
        else:
            intermediate_steps = instance['intermediate_steps']
            for step in intermediate_steps:
                print(instance)
                print(step)
                print(step[-1])
                print('#' * 100)
                if not isinstance(step[-1], str) or not step[-1].startswith('Status Code'):
                    count_not_status_code += 1
                    # print(instance)
                    # print(step)
                    # print(step[-1])
                    # print('#' * 100)
                    if (isinstance(step[-1], str) and ( 
                        'is not a valid action' in step[-1] or 
                        'File does not exist.' in step[-1] or 'Invalid JSON format' in step[-1])):
                        error_not_status_code += 1
                else:
                    response_code = step[-1].split('Status Code: ')[1].split('. Response:')[0]
                    if not 200 <= int(response_code) < 300:
                        error_status_code += 1
                        # print(instance)
                        # print(step)
                        # print(step[-1])
                        # print('#' * 100)
                all_count += 1

{'input': 'Hey, can you show me a random picture of an axolotl?', 'output': 'Here is a random picture of an axolotl: https://theaxolotlapi.netlify.app/images/axolotl5.jpg. It is a beautiful axolotl with unique markings, and is from the Unsplash collection.', 'Final Thought': 'The getRandomAxolotlImage tool successfully retrieved a random axolotl image with information on the image source.', 'intermediate_steps': [[['getRandomAxolotlImage', '{}', ' I can use the getRandomAxolotlImage tool to retrieve a random axolotl image.\nASSISTANT Action: getRandomAxolotlImage\nASSISTANT Action Input: {}'], 'Status Code: 200. Response: {"url":"https://theaxolotlapi.netlify.app/images/axolotl5.jpg","source":"Unsplash","description":"A beautiful axolotl with unique markings"}']]}
[['getRandomAxolotlImage', '{}', ' I can use the getRandomAxolotlImage tool to retrieve a random axolotl image.\nASSISTANT Action: getRandomAxolotlImage\nASSISTANT Action Input: {}'], 'Status Code: 200. Response: {"url":"http

In [11]:
assert error_count + count_not_status_code + count_status_code == all_count

In [10]:
error_status_code

6

In [9]:
error_not_status_code

1

Как будто, тут нет тех же ошибок, что и в оригинале